In [1]:
import networkx as nx
from gerrychain import Graph
import math
import gurobipy as gp
from gurobipy import GRB

#import matplotlib.pyplot as plt
from util import update_attributes, get_k_L_U
from sketch import sketch
from detail import detail
from clusterings import clusterings

filepath1 = 'C:\\districting-data-2020\\'
filepath2 = 'C:\\districting-data-2020-conn\\'

In [2]:
# calculated in enacted_splits.ipynb
enacted_splits = {('AK', 'CD'): 0, ('AL', 'CD'): 6, ('AR', 'CD'): 3, ('AZ', 'CD'): 15, ('CA', 'CD'): 72, ('CO', 'CD'): 20, ('CT', 'CD'): 10, ('DE', 'CD'): 0, ('FL', 'CD'): 31, ('GA', 'CD'): 21, ('HI', 'CD'): 1, ('IA', 'CD'): 0, ('ID', 'CD'): 1, ('IL', 'CD'): 53, ('IN', 'CD'): 8, ('KS', 'CD'): 4, ('KY', 'CD'): 6, ('LA', 'CD'): 15, ('MA', 'CD'): 22, ('MD', 'CD'): 9, ('ME', 'CD'): 1, ('MI', 'CD'): 21, ('MN', 'CD'): 12, ('MO', 'CD'): 10, ('MS', 'CD'): 4, ('MT', 'CD'): 1, ('NC', 'CD'): 13, ('ND', 'CD'): 0, ('NE', 'CD'): 2, ('NH', 'CD'): 5, ('NJ', 'CD'): 20, ('NM', 'CD'): 10, ('NV', 'CD'): 5, ('NY', 'CD'): 26, ('OH', 'CD'): 14, ('OK', 'CD'): 7, ('OR', 'CD'): 16, ('PA', 'CD'): 17, ('RI', 'CD'): 1, ('SC', 'CD'): 10, ('SD', 'CD'): 0, ('TN', 'CD'): 11, ('TX', 'CD'): 59, ('UT', 'CD'): 7, ('VA', 'CD'): 11, ('VT', 'CD'): 0, ('WA', 'CD'): 11, ('WI', 'CD'): 13, ('WV', 'CD'): 0, ('WY', 'CD'): 0, ('AK', 'SS'): 19, ('AL', 'SS'): 35, ('AR', 'SS'): 51, ('AZ', 'SS'): 44, ('CA', 'SS'): 56, ('CO', 'SS'): 42, ('CT', 'SS'): 49, ('DE', 'SS'): 20, ('FL', 'SS'): 32, ('GA', 'SS'): 60, ('HI', 'SS'): 21, ('IA', 'SS'): 46, ('ID', 'SS'): 25, ('IL', 'SS'): 135, ('IN', 'SS'): 48, ('KS', 'SS'): 36, ('KY', 'SS'): 21, ('LA', 'SS'): 77, ('MA', 'SS'): 59, ('MD', 'SS'): 45, ('ME', 'SS'): 40, ('MI', 'SS'): 64, ('MN', 'SS'): 100, ('MO', 'SS'): 16, ('MS', 'SS'): 64, ('MT', 'SS'): 56, ('NC', 'SS'): 24, ('ND', 'SS'): 49, ('NE', 'SS'): 37, ('NH', 'SS'): 40, ('NJ', 'SS'): 56, ('NM', 'SS'): 64, ('NV', 'SS'): 21, ('NY', 'SS'): 66, ('OH', 'SS'): 20, ('OK', 'SS'): 59, ('OR', 'SS'): 47, ('PA', 'SS'): 47, ('RI', 'SS'): 41, ('SC', 'SS'): 68, ('SD', 'SS'): 29, ('TN', 'SS'): 15, ('TX', 'SS'): 41, ('UT', 'SS'): 41, ('VA', 'SS'): 34, ('VT', 'SS'): 18, ('WA', 'SS'): 59, ('WI', 'SS'): 73, ('WV', 'SS'): 13, ('WY', 'SS'): 25, ('AK', 'SH'): 39, ('AL', 'SH'): 115, ('AR', 'SH'): 128, ('AZ', 'SH'): 44, ('CA', 'SH'): 95, ('CO', 'SH'): 73, ('CT', 'SH'): 162, ('DE', 'SH'): 40, ('FL', 'SH'): 112, ('GA', 'SH'): 209, ('HI', 'SH'): 47, ('IA', 'SH'): 92, ('ID', 'SH'): 25, ('IL', 'SH'): 220, ('IN', 'SH'): 129, ('KS', 'SH'): 127, ('KY', 'SH'): 80, ('LA', 'SH'): 116, ('MA', 'SH'): 182, ('MD', 'SH'): 67, ('ME', 'SH'): 166, ('MI', 'SH'): 154, ('MN', 'SH'): 176, ('MO', 'SH'): 137, ('MS', 'SH'): 181, ('MT', 'SH'): 99, ('NC', 'SH'): 80, ('ND', 'SH'): 53, ('NH', 'SH'): 154, ('NJ', 'SH'): 56, ('NM', 'SH'): 86, ('NV', 'SH'): 43, ('NY', 'SH'): 179, ('OH', 'SH'): 77, ('OK', 'SH'): 134, ('OR', 'SH'): 79, ('PA', 'SH'): 186, ('RI', 'SH'): 75, ('SC', 'SH'): 145, ('SD', 'SH'): 31, ('TN', 'SH'): 74, ('TX', 'SH'): 101, ('UT', 'SH'): 72, ('VA', 'SH'): 98, ('VT', 'SH'): 118, ('WA', 'SH'): 59, ('WI', 'SH'): 159, ('WV', 'SH'): 89, ('WY', 'SH'): 56}


In [3]:
def number_of_county_splits(districts_geoid):
    county_geoids = { geoid[0:5] for district in districts_geoid for geoid in district }
    county_assignment = { county_geoid : list() for county_geoid in county_geoids }
    for j in range(len(districts_geoid)):
        for geoid in districts_geoid[j]:
            county_geoid = geoid[0:5]
            if j not in county_assignment[county_geoid]:
                county_assignment[county_geoid].append(j)
    return sum( len(county_assignment[cg]) - 1 for cg in county_assignment.keys() )

In [4]:
import csv

def export_to_baf(geoid_assignment, outfilename):
    
    with open(outfilename, 'w') as csvfile: 
        csvwriter = csv.writer(csvfile) 
        for i in GB.nodes:
            geoid = GB.nodes[i]['GEOID20']
            county = geoid[0:5]
            tract = geoid[0:11]
            block = geoid
            if county in geoid_assignment:
                row = [geoid, geoid_assignment[county]+1]
            elif tract in geoid_assignment:
                row = [geoid, geoid_assignment[tract]+1]
            else:
                row = [geoid, geoid_assignment[block]+1]
            csvwriter.writerow(row) 
    
    return

In [5]:
from number_of_districts import congressional_districts_2020
states = sorted([ state for state in congressional_districts_2020.keys() ])

results = dict()
instances = list( clusterings.keys() )
instances.reverse()

for (state, district_type) in {('WY','SS'),('WY','SH')}: 

    print("**********************************")
    print("State:",state,"district_type:",district_type)
    print("**********************************")

    # get county graph
    filename = state + '_county.json'
    GC = Graph.from_json( filepath1 + filename )
    update_attributes(GC, state)    

    # get tract graph
    filename = state + '_tract.json'
    GT = Graph.from_json( filepath2 + filename )
    update_attributes(GT, state)    

    # get block graph
    filename = state + '_block.json'
    GB = Graph.from_json( filepath2 + filename )
    update_attributes(GB, state)    

    # get values of k, L, U; check connectivity
    (k, L, U) = get_k_L_U(GC, state, district_type)
    if k <= 1 or not nx.is_connected(GC) or not nx.is_connected(GT) or not nx.is_connected(GB):
        print("Skipping this state because k <= 1 or because graph is disconnected.")
        continue

    clusters = clusterings[state,district_type]['clusters'] 
    sizes = clusterings[state,district_type]['sizes'] 
    districts = list()

    for p in range(len(clusters)):

        cluster = clusters[p]
        size = sizes[p]
        GS = GC.subgraph(cluster)
        
        print("size =",size)
        print("cluster =",cluster)
        print("population = ",[ GS.nodes[i]['TOTPOP'] for i in GS.nodes ] )
        print("L,U =",L,U)

        ################################
        # get a sketch for this cluster
        ################################
        
        (xsoln, ysoln, zsoln) = sketch(GS, L, U, size)
        if xsoln is None:
            print("Sketch indicates that this cluster is infeasible; impossible to get k-1 splits!!!")
            districts = None
            break

        ################################
        # visualize this cluster
        ################################
        
#         node_pos = { i : ( GS.nodes[i]['X'], GS.nodes[i]['Y'] ) for i in GS.nodes }
#         node_colors = [ list(xsoln[i])[0] for i in GS.nodes ]
#         edge_colors = [ list(ysoln[u,v])[0] if len(ysoln[u,v]) > 0 else -1 for u,v in GS.edges ]

#         title = state + '_' + district_type
#         plt.figure(title)
#         my_labels = { i : list(xsoln[i].keys()) for i in GS.nodes }
#         nx.draw(GS, pos=node_pos, node_color=node_colors, edge_color=edge_colors, with_labels=True, labels=my_labels, font_color="red")
#         plt.show()

        #########################
        # detail this cluster
        #########################
        
        # first, try tract-level
        sketch_support = { GS.nodes[i]['GEOID20'] : list(xsoln[i].keys()) for i in GS.nodes }
        tracts = [ i for i in GT.nodes if GT.nodes[i]['GEOID20'][0:5] in sketch_support.keys() ]
        cluster_districts = detail(GT.subgraph(tracts), L, U, size, sketch_support)
        cluster_districts_geoid = None # just the initial value!

        if cluster_districts is not None:
            cluster_districts_geoid = [ [ GT.nodes[i]['GEOID20'] for i in district ] for district in cluster_districts ]
        else:
            # second, try block-level
            blocks = [ i for i in GB.nodes if GB.nodes[i]['GEOID20'][0:5] in sketch_support.keys() ]
            cluster_districts = detail(GB.subgraph(blocks), L, U, size, sketch_support)
            if cluster_districts is not None:
                cluster_districts_geoid = [ [ GB.nodes[i]['GEOID20'] for i in district ] for district in cluster_districts ]

        # both tries unsuccessful?
        if cluster_districts_geoid is None:
            districts = None
            break
        else:
            districts += cluster_districts_geoid

    # all clusters successful?
    if districts is not None:
        assert k == len(districts)
        splits = number_of_county_splits(districts)
    else:
        districts_geoid = None
        splits = None

    # store results
    results[state,district_type] = dict()
    results[state,district_type]['clusters'] = clusters
    results[state,district_type]['num_clusters'] = len(sizes)
    results[state,district_type]['sizes'] = sizes
    results[state,district_type]['districts'] = districts
    results[state,district_type]['counties'] = GC.number_of_nodes()
    results[state,district_type]['k'] = k
    results[state,district_type]['L'] = f'{L:,d}'
    results[state,district_type]['U'] = f'{U:,d}'
    results[state,district_type]['obvious_LB'] = sum( math.ceil( GC.nodes[i]['TOTPOP'] / U ) - 1 for i in GC.nodes )
    results[state,district_type]['splits_LB'] = k - len(sizes)
    results[state,district_type]['splits_UB'] = splits
    
    # export districts to block assignment file
    if districts is not None:
        geoid_assignment = { g : j for j in range(len(districts)) for g in districts[j] }
        outfilename = "min_county_splits_" + state + '_' + district_type + "_(by_Maral_Shahmizad_and_Austin_Buchanan_of_Oklahoma_State_University).baf"
        export_to_baf(geoid_assignment, outfilename)  

**********************************
State: WY district_type: SH
**********************************
Starting WY with k = 62 and deviation = 0.1
Thus, we have L = 8839 and U = 9769
size = 11
cluster = [1]
population =  [100512]
L,U = 8839 9769
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-27
Trying tract-level instance.
Iteration 	 Objective 	 iter_time
0 	 0.00 	 0.00
1 	 10897320.42 	 0.00
2 	 7529593.02 	 0.00
3 	 6027280.26 	 0.00
4 	 5900708.36 	 0.00
5 	 5900332.97 	 0.00
6 	 5900324.48 	 0.00
7 	 5900324.48 	 0.00
Set parameter OutputFlag to value 1
Trying DAG model, with roots = [142, 37, 71, 34, 84, 139, 143, 82, 31, 83, 135]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 335 rows, 297 columns and 1969 nonzeros
Model fingerprint: 0xe1e123e1
Va

Presolved: 45449 rows, 41613 columns, 228605 nonzeros
Variable types: 0 continuous, 41613 integer (41613 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 5.580254e+06, 7896 iterations, 0.72 seconds (0.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5580253.58    0   87          - 5580253.58      -     -    3s
H    0     0                    5582734.1764 5580253.58  0.04%     -    4s

Explored 1 nodes (17709 simplex iterations) in 4.10 seconds (1.91 work units)
Thread count was 20 (of 20 available processors)

Solution count 1: 5.58273e+06 

Optimal solution found (tolerance 1.00e-01)
Best objective 5.582734176436e+06, best bound 5.580253575432e+06, gap 0.0444%
size = 9
cluster = [0]
population =  [79955]
L,U = 8839 9769
Trying 

Variable types: 169 continuous, 111 integer (111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [4e+03, 3e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 4 rows and 185 columns
Presolve time: 0.00s
Presolved: 44 rows, 95 columns, 287 nonzeros
Variable types: 0 continuous, 95 integer (95 binary)

Root relaxation: infeasible, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 1 nodes (46 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 20 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 297, time in user-callback 0.01 sec
Trying block-level instance.
Iteration 	 Objective 	 iter_time

     0     0 4.4594e+07    0  245          - 4.4594e+07      -     -    3s
     0     0 4.4594e+07    0  247          - 4.4594e+07      -     -    3s
     0     0 4.4595e+07    0  178          - 4.4595e+07      -     -    4s
H    0     0                    4.475003e+07 4.4595e+07  0.35%     -    4s

Cutting planes:
  Gomory: 11
  Cover: 3
  Implied bound: 10
  Clique: 173
  MIR: 42
  StrongCG: 1
  Zero half: 22
  RLT: 37
  PSD: 1

Explored 1 nodes (7459 simplex iterations) in 4.67 seconds (1.95 work units)
Thread count was 20 (of 20 available processors)

Solution count 1: 4.475e+07 

Optimal solution found (tolerance 1.00e-01)
Best objective 4.475003450155e+07, best bound 4.459464165368e+07, gap 0.3472%
size = 4
cluster = [4]
population =  [37066]
L,U = 8839 9769
Trying tract-level instance.
Iteration 	 Objective 	 iter_time
0 	 0.00 	 0.00
1 	 2649023.79 	 0.00
2 	 2521666.98 	 0.00
3 	 2520325.56 	 0.00
4 	 2520325.56 	 0.00
Set parameter OutputFlag to value 1
Trying DAG model, with

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 14 rows, 24 columns and 72 nonzeros
Model fingerprint: 0x197aab01
Variable types: 6 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [5e+05, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 3 rows and 19 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 46, time in user-callback 0.01 sec
Trying block-level instance.
Iteration 	 Objective 	 iter_time
0 	 0.00 	 0.02
1 	 59853424.23 	 0.01
2 	 32026195.08 	 0.01
3 	 31915089.11 	 0.01
4 	 3191508

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 1 nodes (22 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 20 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 183, time in user-callback 0.00 sec
Trying block-level instance.
Iteration 	 Objective 	 iter_time
0 	 0.00 	 0.04
1 	 28472362.86 	 0.03
2 	 21285253.81 	 0.01
3 	 21261024.89 	 0.01
4 	 21260541.44 	 0.01
5 	 21260524.76 	 0.01
6 	 21260524.76 	 0.01
Set parameter OutputFlag to value 1
Trying DAG model, with roots = [34148, 28246, 6020, 49696, 49318, 6194, 50006]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 43543 rows

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 15 rows, 27 columns and 81 nonzeros
Model fingerprint: 0x5eeae7f9
Variable types: 9 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [4e+05, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 12 rows and 22 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 120, time in user-callback 0.00 sec
Trying block-level instance.
Iteration 	 Objective 	 iter_time
0 	 0.00 	 0.02
1 	 70443351.53 	 0.01
2 	 67676823.95 	 0.01
3 	 67648539.42 	 0.01
4 	 67648

size = 6
cluster = [1, 11]
population =  [100512, 12498]
L,U = 17678 19538
Trying tract-level instance.
Iteration 	 Objective 	 iter_time
0 	 0.00 	 0.00
1 	 40422802.69 	 0.00
2 	 38377542.07 	 0.00
3 	 38349519.44 	 0.00
4 	 38349519.44 	 0.00
Set parameter OutputFlag to value 1
Trying DAG model, with roots = [136, 83, 69, 154, 38, 95]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 223 rows, 186 columns and 1218 nonzeros
Model fingerprint: 0x7d2d5b24
Variable types: 20 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [3e+01, 7e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 110 rows and 83 columns
Presolve time: 0.01s
Presolved: 113 rows, 103 columns, 583 nonzeros
Variable types: 0 c

Presolve time: 1.19s
Presolved: 4274 rows, 2263 columns, 15939 nonzeros
Variable types: 0 continuous, 2263 integer (2257 binary)

Root relaxation: objective 1.293820e+08, 680 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2938e+08    0   44          - 1.2938e+08      -     -    1s
H    0     0                    1.310248e+08 1.2938e+08  1.25%     -    1s

Explored 1 nodes (1217 simplex iterations) in 1.36 seconds (0.58 work units)
Thread count was 20 (of 20 available processors)

Solution count 1: 1.31025e+08 

Optimal solution found (tolerance 1.00e-01)
Best objective 1.310248151023e+08, best bound 1.293819935404e+08, gap 1.2538%
size = 4
cluster = [5, 13, 16]
population =  [4621, 39234, 29624]
L,U = 17678 19538
Trying tract-level instance.
Iteration 	 Objective 	 iter_time
0 	 0.00 	 0.00
1 	 116540406.85 	 0.00
2 	 1086959

In [6]:
print("results =",results)

results = {('WY', 'SH'): {'clusters': [[1], [0], [5, 9, 13, 16], [6, 7, 15, 17], [4], [10, 12], [2, 3, 8, 11, 20], [14, 19], [18, 21], [22]], 'num_clusters': 10, 'sizes': [11, 9, 10, 6, 4, 3, 4, 7, 3, 5], 'districts': [['560210019021080', '560210011002062', '560210019021010', '560210011002065', '560210019022135', '560210019021171', '560210019021064', '560210019021059', '560210012001007', '560210019023156', '560210019022074', '560210012002020', '560210011002029', '560210019022041', '560210019022045', '560210019022077', '560210019023212', '560210019021154', '560210019022111', '560210019023197', '560210019021149', '560210011002024', '560210019022185', '560210012001035', '560210019022088', '560210019021191', '560210019021040', '560210011003010', '560210019023107', '560210019022044', '560210019023126', '560210019022063', '560210012001048', '560210019021151', '560210019021024', '560210019021132', '560210012001029', '560210019023173', '560210019023074', '560210012001034', '560210019023059', '

In [7]:
print('state  district_type  splits_LB  splits_UB  gap')
for (state, district_type) in results.keys():
    splits_LB = results[state,district_type]['k'] - clusterings[state,district_type]['cluster_UB']
    splits_UB = results[state,district_type]['splits_UB'] 
    if splits_UB is None:
        print(state,district_type, splits_LB, splits_UB, 'None')
    else:
        print(state,district_type, splits_LB, splits_UB, splits_UB-splits_LB)

state  district_type  splits_LB  splits_UB  gap
WY SH 52 52 0
WY SS 22 22 0


In [8]:
for district_type in ['CD', 'SS', 'SH']:
    print("results for",district_type,":")
    print("state |C| k L U obvious_LB max_clusters split_LB min_splits enacted_splits")
    for state in states:
        if (state, district_type) in results.keys():
            s = state
            t = district_type
            print(state,end=' & ')
            print(results[s,t]['counties'],end=' & ')
            print(results[s,t]['k'],end=' & ')
            print(results[s,t]['L'],end=' & ')
            print(results[s,t]['U'],end=' & ')
            print(results[s,t]['obvious_LB'],end=' & ')
            print(results[s,t]['num_clusters'],end=' & ')
            print(results[s,t]['splits_LB'],end=' & ')
            print(results[s,t]['splits_UB'],end=' & ')
            print(enacted_splits[s,t],'\\\\')  
            
    print("\n\n")

results for CD :
state |C| k L U obvious_LB max_clusters split_LB min_splits enacted_splits



results for SS :
state |C| k L U obvious_LB max_clusters split_LB min_splits enacted_splits
WY & 23 & 31 & 17,678 & 19,538 & 21 & 9 & 22 & 22 & 25 \\



results for SH :
state |C| k L U obvious_LB max_clusters split_LB min_splits enacted_splits
WY & 23 & 62 & 8,839 & 9,769 & 49 & 10 & 52 & 52 & 56 \\



